In [1]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('./train_cascade_metrics_Approach_B.csv')

In [7]:
# df2 = pd.read_csv('./train_cascade_metrics_Approach_C_2.csv')

In [8]:
# df3 = pd.read_csv('./train_cascade_metrics_Approach_C_3.csv')

In [9]:
# df = pd.concat([df1, df2, df3])

In [10]:
df.reset_index(inplace = True)

In [11]:
df.head(3)

,index,seed_file_name,total_distincet_infected_nodes,total_males_in_casc,total_females_in_cas,males_fraction_in_casc,females_fraction_in_casc,fairness_score_value
0,0,../Data/Weibo/Seeds/final_seeds_5g_05a_200s_0_...,907208,391114,516094,0.4311,0.5689,0.3859
1,1,../Data/Weibo/Seeds/final_seeds_5g_05a_500s_1b...,990608,432131,558477,0.4362,0.5638,0.3940
2,2,../Data/Weibo/Seeds/final_seeds_5g_05a_100s_1b...,809195,336927,472268,0.4164,0.5836,0.3633


In [12]:
df.tail(3)

,index,seed_file_name,total_distincet_infected_nodes,total_males_in_casc,total_females_in_cas,males_fraction_in_casc,females_fraction_in_casc,fairness_score_value
27,27,../Data/Weibo/Seeds/final_seeds_5g_05a_400s_0_...,955280,414322,540958,0.4337,0.5663,0.3900
28,28,../Data/Weibo/Seeds/final_seeds_5g_05a_500s_0_...,958902,415636,543266,0.4334,0.5666,0.3896
29,29,../Data/Weibo/Seeds/final_seeds_5g_05a_100s_7b...,652261,276148,376113,0.4234,0.5766,0.3738


In [13]:
def get_seed_length(filename):
    filename_list = filename.split('_')
    seed_length = int(filename_list[4].split('s')[0])
    return seed_length
get_seed_length('../Data/Weibo/Seeds/final_seeds_5g_05a_750s_5b.txt')    

750

In [14]:
df['seed_length'] = df['seed_file_name'].apply(get_seed_length)
df.head(3)

,index,seed_file_name,total_distincet_infected_nodes,total_males_in_casc,total_females_in_cas,males_fraction_in_casc,females_fraction_in_casc,fairness_score_value,seed_length
0,0,../Data/Weibo/Seeds/final_seeds_5g_05a_200s_0_...,907208,391114,516094,0.4311,0.5689,0.3859,200
1,1,../Data/Weibo/Seeds/final_seeds_5g_05a_500s_1b...,990608,432131,558477,0.4362,0.5638,0.3940,500
2,2,../Data/Weibo/Seeds/final_seeds_5g_05a_100s_1b...,809195,336927,472268,0.4164,0.5836,0.3633,100


## Train Max Bound:

```
TRAIN CASCADE: Max bound on infectors by seed size 100 = 925411
TRAIN CASCADE: Max bound on infectors by seed size 200 = 983485
TRAIN CASCADE: Max bound on infectors by seed size 300 = 1011755
TRAIN CASCADE: Max bound on infectors by seed size 400 = 1029062
TRAIN CASCADE: Max bound on infectors by seed size 500 = 1041105

TRAIN CASCADE: Max bound on fairness by seed size 100 is 0.9969 with 39160 infectors
TRAIN CASCADE: Max bound on fairness by seed size 200 is 0.9917 with 163816 infectors
TRAIN CASCADE: Max bound on fairness by seed size 300 is 0.9941 with 207317 infectors
TRAIN CASCADE: Max bound on fairness by seed size 400 is 0.985 with 272893 infectors
TRAIN CASCADE: Max bound on fairness by seed size 500 is 0.9887 with 322362 infectors
```

In [15]:
train_seed_max_bound_dict = {
    '100': [925411, 0.9969],
    '200': [983485, 0.9917],
    '300': [1011755, 0.9941],
    '400': [1029062, 0.985],       
    '500': [1041105, 0.9887]
}

## Test Max Bound:

```
TEST CASCADE: Max bound on infectors by seed size 100 = 429435
TEST CASCADE: Max bound on infectors by seed size 200 = 496695
TEST CASCADE: Max bound on infectors by seed size 300 = 531831
TEST CASCADE: Max bound on infectors by seed size 400 = 554362
TEST CASCADE: Max bound on infectors by seed size 500 = 569986

TEST CASCADE: Max bound on fairness by seed size 100 is 0.992 with 57226 infectors
TEST CASCADE: Max bound on fairness by seed size 200 is 0.9976 with 79471 infectors
TEST CASCADE: Max bound on fairness by seed size 300 is 0.9883 with 117896 infectors
TEST CASCADE: Max bound on fairness by seed size 400 is 0.9824 with 137874 infectors
TEST CASCADE: Max bound on fairness by seed size 500 is 0.9998 with 188638 infectors
```

In [16]:
test_seed_max_bound_dict = {
    '100': [429435, 0.992],
    '200': [496695, 0.9976],
    '300': [531831, 0.9883],
    '400': [554362, 0.9824],       
    '500': [569986, 0.9998]
}

In [17]:
df.columns

Index(['index', 'seed_file_name', 'total_distincet_infected_nodes',
       'total_males_in_casc', 'total_females_in_cas', 'males_fraction_in_casc',
       'females_fraction_in_casc', 'fairness_score_value', 'seed_length'],
      dtype='object')

In [21]:
def get_objective_value(total_distincet_infected_nodes, fairness_score_value, seed_length):
    max_fairness = train_seed_max_bound_dict[str(seed_length)][1]
    max_infected = train_seed_max_bound_dict[str(seed_length)][0]    
    fairness_frac = fairness_score_value / max_fairness
    infected_frac = total_distincet_infected_nodes / max_infected
#     print(max_infected, max_fairness,  fairness_frac, infected_frac )
    objective_value = (fairness_frac + infected_frac)/2
    return np.round(objective_value, 4)

print(get_objective_value(925411, 0.9969, '100'))
print(get_objective_value(925411/2, 0.9969/2, '100'))

1.0
0.5


In [22]:
# df['objective_value'] = df[['total_distincet_infected_nodes', 'fairness_score_value', 'seed_length']].apply(get_objective_value, axis = 1)
df['objective_value'] = df.apply(lambda x: get_objective_value(x.total_distincet_infected_nodes, x.fairness_score_value, x.seed_length), axis=1)
df.head(3)

,index,seed_file_name,total_distincet_infected_nodes,total_males_in_casc,total_females_in_cas,males_fraction_in_casc,females_fraction_in_casc,fairness_score_value,seed_length,objective_value
0,0,../Data/Weibo/Seeds/final_seeds_5g_05a_200s_0_...,907208,391114,516094,0.4311,0.5689,0.3859,200,0.6558
1,1,../Data/Weibo/Seeds/final_seeds_5g_05a_500s_1b...,990608,432131,558477,0.4362,0.5638,0.3940,500,0.6750
2,2,../Data/Weibo/Seeds/final_seeds_5g_05a_100s_1b...,809195,336927,472268,0.4164,0.5836,0.3633,100,0.6194


In [23]:
df2 = df.sort_values(['seed_length', 'objective_value'])
df2

,index,seed_file_name,total_distincet_infected_nodes,total_males_in_casc,total_females_in_cas,males_fraction_in_casc,females_fraction_in_casc,fairness_score_value,seed_length,objective_value
24,24,../Data/Weibo/Seeds/final_seeds_5g_05a_100s_5b...,401989,174961,227028,0.4352,0.5648,0.3924,100,0.4140
29,29,../Data/Weibo/Seeds/final_seeds_5g_05a_100s_7b...,652261,276148,376113,0.4234,0.5766,0.3738,100,0.5399
11,11,../Data/Weibo/Seeds/final_seeds_5g_05a_100s_0_...,776641,316629,460012,0.4077,0.5923,0.3505,100,0.5954
2,2,../Data/Weibo/Seeds/final_seeds_5g_05a_100s_1b...,809195,336927,472268,0.4164,0.5836,0.3633,100,0.6194
9,9,../Data/Weibo/Seeds/final_seeds_5g_05a_100s_0_...,829843,353109,476734,0.4255,0.5745,0.3771,100,0.6375
23,23,../Data/Weibo/Seeds/final_seeds_5g_05a_100s_3b...,838891,358440,480451,0.4273,0.5727,0.3799,100,0.6438
12,12,../Data/Weibo/Seeds/final_seeds_5g_05a_200s_0_...,251001,111387,139614,0.4438,0.5562,0.4062,200,0.3324
13,13,../Data/Weibo/Seeds/final_seeds_5g_05a_200s_7b...,596158,261342,334816,0.4384,0.5616,0.3974,200,0.5034
18,18,../Data/Weibo/Seeds/final_seeds_5g_05a_200s_5b...,660213,286008,374205,0.4332,0.5668,0.3892,200,0.5319
26,26,../Data/Weibo/Seeds/final_seeds_5g_05a_200s_3b...,874165,378428,495737,0.4329,0.5671,0.3887,200,0.6404


In [25]:
df2.to_csv('./sorted_best_train_objective_values_approach_B.csv', index = False)